In [125]:
import tensorflow as tf
import sounddevice as sd
import soundfile as sf
import numpy as np


In [126]:

interpreter = tf.lite.Interpreter(model_path="soundclassifier_with_metadata.tflite")
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)
sample_rate =44100
duration=30
frames = sample_rate * duration
audio_buffer = np.array([])

[{'name': 'audio_preproc_input', 'index': 0, 'shape': array([    1, 44032]), 'shape_signature': array([   -1, 44032]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 75, 'shape': array([1, 7]), 'shape_signature': array([-1,  7]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [127]:
def preprocess_audio(audio_data):
    audio_data = np.array(audio_data)

    audio_data = audio_data / np.max(np.abs(audio_data))

    audio_data = audio_data.reshape(input_details[0]['shape'])

    return audio_data

def interpret_output(output_data):
    class_labels = ['Aplaudiendo', 'Cantando 1', 'Cantando 2', 'Hablando', 'Sonido ambiente', 'silvando', 'tosiendo']

    highest_output_index = np.argmax(output_data)

    highest_output_label = class_labels[highest_output_index]

    print(f"El sonido es: {highest_output_label}")
    
def audio_callback(indata, frames, time, status):
    global audio_buffer

    audio_buffer = np.concatenate((audio_buffer, indata.flatten()))
    

    if audio_buffer.size >= 44032:
        audio_data = audio_buffer[:44032]

        audio_data = audio_data / np.max(np.abs(audio_data))

        audio_data = audio_data.astype(np.float32)

        audio_data = audio_data.reshape(input_details[0]['shape'])

        interpreter.set_tensor(input_details[0]['index'], audio_data)

        interpreter.invoke()

        output_data = interpreter.get_tensor(output_details[0]['index'])

        interpret_output(output_data)
        
        audio_buffer = audio_buffer[44032:]




In [128]:
with sd.InputStream(samplerate=sample_rate, channels=1, callback=audio_callback):
    sd.sleep(duration * 1000)

El sonido es: Sonido ambiente
El sonido es: Sonido ambiente
El sonido es: silvando
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Hablando
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Sonido ambiente
El sonido es: Cantando 2
El sonido es: Cantando 1
El sonido es: Cantando 2
El sonido es: Cantando 1
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Sonido ambiente
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Cantando 2
El sonido es: Aplaudiendo
El sonido es: Cantando 1
El sonido es: Cantando 2
El sonido es: tosiendo
El sonido es: Cantando 2
